In [1]:
from fastai2.basics import *
from fastai2.vision.all import *
from fastai2.callback.all import *

In [2]:
path = untar_data(URLs.IMAGEWOOF); path.ls()

(#3) [/home/jupyter/.fastai/data/imagewoof/models,/home/jupyter/.fastai/data/imagewoof/val,/home/jupyter/.fastai/data/imagewoof/train]

In [3]:
dblock = DataBlock(blocks=(ImageBlock, ImageBlock),
                   get_items=get_image_files,
                   splitter=GrandparentSplitter(valid_name='val'),
                   get_y=lambda x: x,
                 )

In [4]:
bs = 32
dbch = dblock.databunch(path, bs=bs, item_tfms=[Resize(128)], batch_tfms=[*aug_transforms(), Normalize(*imagenet_stats)])
dbch.c = 3

In [5]:
# How can I be sure this is working correctly?
pipe = Pipeline([PILImage.create, Resize(128), ToTensor(), IntToFloatTensor(), Normalize(*imagenet_stats, cuda=False)])
style_timg = TensorImage(pipe('styles/udnie.jpg').cuda())

In [6]:
def gram_matrix(x):
    n,c,h,w = x.size()
    x = x.view(n, c, -1)
    return (x @ x.transpose(1,2))/(c*h*w)
def gram(x):
    n,c,h,w = x.size()
    x = x.view(n, c, -1)
    return (x @ x.transpose(1,2))/(c*h*w)

In [7]:
base_loss = F.l1_loss
class FeatureLoss(nn.Module):
    def __init__(self, m_feat, layers, layer_wgts):
        super().__init__()
        self.m_feat = m_feat
        self.loss_features = layers
        self.hooks = hook_outputs(self.loss_features, detach=False)
        self.wgts = layer_wgts
        self.metric_names = ['pixel',] + [f'feat_{i}' for i in range(len(layers))
              ] + [f'gram_{i}' for i in range(len(layers))]

    def make_features(self, x, clone=False):
        self.m_feat(x)
        return [(o.clone() if clone else o) for o in self.hooks.stored]
    
    def forward(self, input, target):
        out_feat = self.make_features(target, clone=True)
        in_feat = self.make_features(input)
        self.feat_losses = [base_loss(input,target)]
        self.feat_losses += [base_loss(f_in, f_out)*w
                             for f_in, f_out, w in zip(in_feat, out_feat, self.wgts)]
        self.feat_losses += [base_loss(gram_matrix(f_in), gram_matrix(f_out))*w**2 * 5e3
                             for f_in, f_out, w in zip(in_feat, out_feat, self.wgts)]
        self.metrics = dict(zip(self.metric_names, self.feat_losses))
        return sum(self.feat_losses)
    
    def __del__(self): self.hooks.remove()

In [8]:
vgg_m = vgg16_bn(True).features.cuda().eval()
for p in vgg_m.parameters(): p.requires_grad = False
requires_grad(vgg_m)

False

In [9]:
blocks = [i-1 for i,o in enumerate(vgg_m.children()) if isinstance(o,nn.MaxPool2d)]
layers = [vgg_m[i] for i in blocks]

In [10]:
feat_loss = FeatureLoss(vgg_m, layers=layers[2:5], layer_wgts=[5, 15, 2])

In [11]:
config = dict(blur=True, self_attention=True, y_range=(-3, 3), norm_type=NormType.Batch)
learn = unet_learner(dbch, partial(xresnet50, sa=True), loss_func=feat_loss, config=config)

In [12]:
xb, yb = dbch.one_batch()

In [13]:
pred = learn.model(xb)

In [21]:
# Features of style img, only needs to be computed once
style_fs = feat_loss.make_features(style_timg)
style_fs = [o.repeat(bs,1,1,1) for o in style_fs]; style_fs[0].shape

torch.Size([32, 256, 32, 32])

In [20]:
pred_fs = feat_loss.make_features(pred); pred_fs[0].shape

torch.Size([32, 256, 32, 32])

In [24]:
w = 1 # Hardcoded, should be a list of for each layer
style_loss = [base_loss(gram(pred_f), gram(style_f)*w**2 * 5e3) for pred_f,style_f in zip(pred_fs,style_fs)]

In [25]:
targ_fs = feat_loss.make_features(yb); targ_fs[0].shape

torch.Size([32, 256, 32, 32])

In [26]:
pred.shape, yb.shape

(torch.Size([32, 3, 128, 128]), torch.Size([32, 3, 128, 128]))

In [31]:
w = 1 # Hardcoded, should be a list of for each layer
cnt_loss =  [base_loss(pred,yb)]
cnt_loss += [base_loss(pred_f,targ_f)*w for pred_f,targ_f in zip(pred_fs, targ_fs)]
cnt_loss += [base_loss(gram(pred_f),gram(targ_f))*w for pred_f,targ_f in zip(pred_fs, targ_fs)]
sum(cnt_loss)

tensor(1.5958, device='cuda:0', grad_fn=<AddBackward0>)

In [38]:
Hook??